# Yolo Model

### Imports 

In [1]:
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 2.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
! yolo mode=checks

Traceback (most recent call last):
  File "/home/sdelannoy/.pyenv/versions/3.10.6/envs/foodbuddys/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/home/sdelannoy/.pyenv/versions/3.10.6/envs/foodbuddys/lib/python3.10/site-packages/ultralytics/cfg/__init__.py", line 909, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/home/sdelannoy/.pyenv/versions/3.10.6/envs/foodbuddys/lib/python3.10/site-packages/ultralytics/utils/checks.py'>'. Valid modes are {'val', 'benchmark', 'train', 'track', 'predict', 'export'}.

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'segment', 'obb', 'detect', 'classify', 'pose'}
                MODE (required) is one of {'val', 'benchmark', 'train', 'track', 'predict', 'export'}
                ARGS (

### Dataset Pre Processing

In [3]:
import json
from collections import defaultdict
import random

# Charger le fichier JSON
with open("../raw_data/public_training_set_release_2.0/annotations.json", "r") as f:
    data = json.load(f)

# Compter le nombre d'annotations par image
image_annotation_count = defaultdict(int)
for annotation in data["annotations"]:
    image_annotation_count[annotation["image_id"]] += 1

# Filtrer les images avec 1 ou 2 annotations
filtered_image_ids = {image_id for image_id, count in image_annotation_count.items() if count in [1, 2]}

# Ajouter 20 images avec 3 annotations
three_annotation_images = [image_id for image_id, count in image_annotation_count.items() if count == 3]
filtered_image_ids.update(random.sample(three_annotation_images, min(100, len(three_annotation_images))))

# Ajouter 20 images avec 4 annotations
four_annotation_images = [image_id for image_id, count in image_annotation_count.items() if count == 4]
filtered_image_ids.update(random.sample(four_annotation_images, min(50, len(four_annotation_images))))

# Créer les nouvelles sections filtrées
filtered_images = [img for img in data["images"] if img["id"] in filtered_image_ids]
filtered_annotations = [anno for anno in data["annotations"] if anno["image_id"] in filtered_image_ids]

# Conserver uniquement les catégories utilisées
used_category_ids = {anno["category_id"] for anno in filtered_annotations}
filtered_categories = [cat for cat in data["categories"] if cat["id"] in used_category_ids]

# Construire le nouveau dataset
filtered_data = {
    "images": filtered_images,
    "annotations": filtered_annotations,
    "categories": filtered_categories,
    "info": data.get("info", {}),
    "licenses": data.get("licenses", [])
}

# Sauvegarder le nouveau fichier JSON
with open("../raw_data/public_training_set_release_2.0/filtered_annotations.json", "w") as f:
    json.dump(filtered_data, f)

print(f"Dataset filtré avec {len(filtered_images)} images, dont :")
print(f"- {len([img for img in filtered_images if image_annotation_count[img['id']] == 1])} avec 1 annotation")
print(f"- {len([img for img in filtered_images if image_annotation_count[img['id']] == 2])} avec 2 annotations")
print(f"- 100 avec 3 annotations")
print(f"- 50 avec 4 annotations")

Dataset filtré avec 31166 images, dont :
- 23134 avec 1 annotation
- 7882 avec 2 annotations
- 100 avec 3 annotations
- 50 avec 4 annotations


In [4]:
import os
import shutil
import json

# Chemins des dossiers et fichiers
annotations_path = "../raw_data/public_training_set_release_2.0/filtered_annotations.json"
images_path = "../raw_data/public_training_set_release_2.0/images/"  
output_path = "../raw_data/public_training_set_release_2.0/images_filtered/"

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_path, exist_ok=True)

# Charger le fichier JSON
with open(annotations_path, "r") as f:
    annotations = json.load(f)

# Extraire les noms des fichiers nécessaires
filtered_files = {img["file_name"] for img in annotations["images"]}

# Parcourir le dossier images et copier uniquement les fichiers nécessaires
for file_name in os.listdir(images_path):
    if file_name in filtered_files:
        src = os.path.join(images_path, file_name)
        dest = os.path.join(output_path, file_name)
        shutil.copy(src, dest)

print(f"Images filtrées copiées dans : {output_path}")

Images filtrées copiées dans : ../raw_data/public_training_set_release_2.0/images_filtered/
